# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\tangy\AppData\Local\Temp\graphlab_server_1489851417.log.0


This non-commercial license of GraphLab Create for academic use is assigned to yifei.tang@uconn.edu and will expire on February 03, 2018.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will **increase the separation** between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will **decrease the separation** between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None,
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.279923     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.299975     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.coefficients[model_all.coefficients['value']!=0].print_rows(num_rows=30)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
+------------------+-------+---------------+--------+
[6 rows x 4 columns]



Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

**Ans: ** bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
import numpy as np
l1_penalty = np.logspace(1, 7, num=13)
print (l1_penalty)

[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


In [9]:
temp = 1e20
for i in l1_penalty:
        model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=i)
        rss = sum((model.predict(validation) - validation['price'])**2)
        if rss < temp:
            temp = rss
            l1 = i
print l1

10.0


Also, using this value of L1 penalty, how many nonzero weights do you have?

In [10]:
model_l1_10 = graphlab.linear_regression.create(sales, target='price', features=all_features, validation_set=None, verbose = False,
                                                l2_penalty=0., l1_penalty=10)
model_l1_10.coefficients[model.coefficients['value']!=0].print_rows(num_rows=30)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  20459.2475219   |  None  |
|     bedrooms     |  None |  8155.38098737   |  None  |
| bedrooms_square  |  None |  1479.73787423   |  None  |
|    bathrooms     |  None |  24576.2383172   |  None  |
|   sqft_living    |  None |  37.4911504798   |  None  |
| sqft_living_sqrt |  None |  1109.39597073   |  None  |
|     sqft_lot     |  None | -0.0168499198461 |  None  |
|  sqft_lot_sqrt   |  None |  149.569423985   |  None  |
|      floors      |  None |  20983.5137368   |  None  |
|  floors_square   |  None |  12278.1023451   |  None  |
|    waterfront    |  None |  581971.306649   |  None  |
|       view       |  None |  92988.9899686   |  None  |
|    condition     |  None |  6924.28719657   |  None  |
|      grade       |  None |  6205.64105779   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- **an interpretable model that has only a few features in them**.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [11]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [12]:
l1_penalty_values = np.logspace(8, 10, num=20)
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [13]:
temp = 1e20
numPara = []
for i in xrange(20):
        model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=l1_penalty_values[i])
        numPara.insert(i, model['coefficients']['value'].nnz())

In [14]:
numPara.sort(reverse=True)
print(numPara)

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min`
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` 


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [15]:
for i in range(20):
    if numPara[i] < max_nonzeros:
        break
l1_penalty_min = l1_penalty_values[i-1]
l1_penalty_max = l1_penalty_values[i]
print(i)
print 'l1_penalty_min:', l1_penalty_min
print 'l1_penalty_max:', l1_penalty_max

15
l1_penalty_min: 2976351441.63
l1_penalty_max: 3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

l1_penalty_min: 2976351441.63; l1_penalty_max: 3792690190.73

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [16]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
print l1_penalty_values

[  2.97635144e+09   3.01931664e+09   3.06228184e+09   3.10524703e+09
   3.14821223e+09   3.19117743e+09   3.23414263e+09   3.27710782e+09
   3.32007302e+09   3.36303822e+09   3.40600341e+09   3.44896861e+09
   3.49193381e+09   3.53489901e+09   3.57786420e+09   3.62082940e+09
   3.66379460e+09   3.70675980e+09   3.74972499e+09   3.79269019e+09]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [17]:
rss = 1e50
#l1_optimal = 0
for i in range(20):
        model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=l1_penalty_values[i])
        if model['coefficients']['value'].nnz() == max_nonzeros:
            temp = sum((model.predict(validation) - validation['price'])**2)
            if temp < rss:
                l1_optimal = i

In [18]:
print l1_optimal
print l1_penalty_values[l1_optimal]

14
3577864204.13


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [19]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose = False,
                                                  l2_penalty=0., l1_penalty=l1_penalty_values[l1_optimal])
model.coefficients[model.coefficients['value']!=0].print_rows(num_rows=30)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 229077.875448 |  None  |
|     bedrooms     |  None | 149.417508491 |  None  |
|    bathrooms     |  None | 15146.7392756 |  None  |
|   sqft_living    |  None | 31.7586377842 |  None  |
| sqft_living_sqrt |  None |  655.07824878 |  None  |
|      grade       |  None | 2670.25064395 |  None  |
|    sqft_above    |  None | 29.0979663052 |  None  |
+------------------+-------+---------------+--------+
[7 rows x 4 columns]

